In [ ]:
!pip install mlflow

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tensorflow import keras
import mlflow

import pandas as pd
from plotly import express as px
from plotly import graph_objects as go

# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D, MaxPool2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
tf.__version__# Set your variables for your environment


In [ ]:
import os
import glob
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/butterfly-images40-species/butterflies and moths.csv")
df.head()

In [ ]:
# df = df.iloc[:3000,:]

In [ ]:
width , height = (299, 299)
n_classes = df['labels'].nunique()
print(n_classes)
df.columns

In [ ]:
#preprocessing data :
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(#rotation_range=90,
                                                                brightness_range=(0.5,1),
                                                                #shear_range=0.2,
                                                                #zoom_range=0.2,
                                                                channel_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                rescale=1./255,
                                                                validation_split=0.3)


img_generator_flow_train = img_generator.flow_from_dataframe(
    dataframe=df, # the dataframe containing the filename and label column
    directory="/kaggle/input/butterfly-images40-species", # the directory containing the image files
    x_col="filepaths", # the name of the column with the filenames
    y_col="labels", # the name of the column with the labels
    target_size=(width, height),
    class_mode = "categorical", 
    batch_size=32, 
    subset="training")

img_generator_flow_valid = img_generator.flow_from_dataframe(
    dataframe=df, # the dataframe containing the filename and label column
    directory="/kaggle/input/butterfly-images40-species", # the directory containing the image files
    x_col="filepaths", # the name of the column with the filenames
    y_col="labels", # the name of the column with the labels
    target_size=(width, height),
    class_mode = "categorical", 
    batch_size=32, 
    subset="validation")

In [ ]:
# Set your variables for your environment
EXPERIMENT_NAME="papillons-elo"

# Set tracking URI to your Heroku application
# mlflow.set_tracking_uri(os.environ["APP_URI"])
mlflow.set_tracking_uri("https://insects-mlflow.herokuapp.com/")


# Set experiment's info 
mlflow.set_experiment(EXPERIMENT_NAME)



# Get our experiment info
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

In [ ]:
# Call mlflow autolog
mlflow.tensorflow.autolog()

mlflow.end_run()

with mlflow.start_run(experiment_id = experiment.experiment_id):
   
    mlflow.set_tag("user", "elo")
    mlflow.set_tag("model", "inceptionv3")
    mlflow.set_tag("description", "24cat")
    
    base_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(width, height,3),
                                               include_top=False,
                                               weights = "imagenet"
                                               )
    base_model.trainable = True

    fine_tune_at = len(base_model.layers) - 0.1*len(base_model.layers)

    for layer in base_model.layers[:int(fine_tune_at)]:
        layer.trainable = False
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalMaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(n_classes, activation="softmax")
    ])
    model.summary()

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
                  loss = tf.keras.losses.CategoricalCrossentropy(),
                  metrics = [tf.keras.metrics.CategoricalAccuracy()])
    model.fit(img_generator_flow_train, validation_data=img_generator_flow_valid, epochs=100)

In [ ]:
# Visualise train / Valid Accuracy
plt.plot(model.history.history["categorical_accuracy"], c="r", label="train_accuracy")
plt.plot(model.history.history["val_categorical_accuracy"], c="b", label="test_accuracy")
plt.legend(loc="upper left")
plt.show()



In [ ]:
# Visualise train / Valid Loss
plt.plot(model.history.history["loss"], c="r", label="train_loss")
plt.plot(model.history.history["val_loss"], c="b", label="test_loss")
plt.legend(loc="upper left")
plt.show()
